# CS295/395: Secure Distributed Computation
## Homework 3

## Definitions

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import urllib.request

_PRIME = 2 ** 13 - 1

shamir_lib_url = "https://raw.githubusercontent.com/jnear/cs295-secure-computation/master/utils/shamir.py"

### DANGER: this line is dangerous. Make sure the URL above is correct, and has correct code.
exec(urllib.request.urlopen(shamir_lib_url).read())

def share_shamir(t, n, x, prime=_PRIME):
    shares_with_x = share_input(x, minimum=t, shares=n, prime=prime)
    return [y for x,y in shares_with_x]

def reconstruct_shamir(shares, prime=_PRIME):
    shares_with_x = list(zip(range(1, len(shares)+1), shares))
    return recover_secret(shares_with_x, prime=prime)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self, field_size=_PRIME):
        """Initialize the field size and dictionary to hold received messages."""
        self.field_size = field_size
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

In [ ]:
reconstruct_shamir(share_shamir(3, 5, 250))

## Question 1 (20 points)

Implement a protocol that calculates the *mean* of the parties' input numbers, secure against semi-honest adversaries. The specification for the `mean` function is below.

In [ ]:
nums = [np.random.randint(0, 200) for _ in range(10)]

def mean(nums):
    s = np.sum(nums)
    return s/len(nums)

mean(nums)

### Question 1(a)

Write an English description of the protocol, in the same format as the protocol definition from our exercise on 9/11. The ideal functionality is as follows:

\begin{equation*}
\textbf{Functionality: Mean}\\
\fbox{$\mathcal{F}_{mean}(x_1, \dots, x_n) = \frac{1}{n} \sum_{i=1}^n x_i$}
\end{equation*}

*HINT*: think about how to implement `mean` with addition *only*. Keep in mind that the number of parties is public knowledge (so it could be used to post-process the protocol's result).

YOUR ANSWER HERE

### Question 1(b)

Implement your protocol from question 1(a), using Shamir secret sharing.

In [ ]:
class MeanParty(Party):
    def round1(self, parties, input_num):
        self.input = input_num
        self.parties = parties
        n = len(parties)
        t = n-1
        # YOUR CODE HERE
        raise NotImplementedError()

    def round2(self):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round3(self):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE for question 1(b)

NUM_PARTIES = 10

parties = [MeanParty() for _ in range(NUM_PARTIES)]

nums = [np.random.randint(0, 10) for _ in range(NUM_PARTIES)]

for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    print(p.get_view())
    assert np.abs(p.output - np.mean(nums)) < 1

## Question 2 (20 points)

Implement a protocol that calculates the *variance* of the parties' input numbers, secure against semi-honest adversaries. The specification for the `variance` function is below. Your protocol **may leak the mean of the numbers**.

In [ ]:
def variance(nums):
    m = mean(nums)
    sq = [(x - m)**2 for x in nums]
    ssum = np.sum(sq)
    return ssum/len(nums)

variance(nums)

### Question 2(a)

Write an English description of the protocol, in the same format as the protocol definition from our exercise on 9/11.

\begin{equation*}
\textbf{Functionality: Variance}\\
\fbox{$\mathcal{F}_{var}(x_1, \dots, x_n) = \frac{\sum_{i=1}^n (x_i - \mu)^2}{n}$}\\
\text{where}\;\; \mu = \frac{1}{n} \sum_{i=1}^n x_i
\end{equation*}

*HINT*: Calculating the variance requires calculating the mean first. Start with your answer to 1(a), and extend it to calculate the variance with additional rounds. These rounds only require addition. As in question 1, the number of parties is public knowledge (so it could be used to post-process the protocol's result).

YOUR ANSWER HERE

### Question 2(b)

Implement your protocol from question 2(a), using Shamir secret sharing. *HINT*: start with your solution to 1(b) and extend it.

In [ ]:
class VarianceParty(Party):
    def round1(self, parties, input_num):
        self.input = input_num
        self.parties = parties
        n = len(parties)
        t = n-1
        # YOUR CODE HERE
        raise NotImplementedError()

    def round2(self):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round3(self):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round4(self):
        # YOUR CODE HERE
        raise NotImplementedError()

    def round5(self):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE for question 2(b)
NUM_PARTIES = 10

parties = [VarianceParty() for _ in range(NUM_PARTIES)]

nums = [np.random.randint(0, 50) for _ in range(NUM_PARTIES)]

for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    p.round4()
for p in parties:
    p.round5()
for p in parties:
    #print(p.get_view())
    assert np.abs(p.output - np.var(nums)) <= 1

## Question 3 (20 points)

Implement a protocol to *multiply* three input numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive one share (see the test case for details). The protocol is defined as follows:

\begin{equation*}
\textbf{Functionality: Multiply Three Numbers}\\
\fbox{$\mathcal{F}(a, b, c) = a \cdot b \cdot c$}
\end{equation*}


**Protocol: Multiplication with Shamir Secret Sharing**
- **Round 1**: Each party $P_i$ receives shares $a_i, b_i, c_i$ as input. $P_i$ performs finite field multiplication to calculate $s_i = a_i \times b_i \times c_i$, a share of the quantity $a\cdot b \cdot c$. $P_i$ sends $s_i$ to all of the parties.
- **Round 2**: Each party $P_i$ receives shares $s_1, \dots, s_n$ of the product. $P_i$ reconstructs the value from these shares and outputs the value.

In [ ]:
class MultThreeParty(Party):
    def round1(self, parties, a_shr, b_shr, c_shr):
        self.input = (a_shr, b_shr, c_shr)
        
        # YOUR CODE HERE
        raise NotImplementedError()

    def round2(self):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
NUM_PARTIES = 10
# (t, n)-Shamir scheme
n = NUM_PARTIES
t = 3

shares1 = share_shamir(t, n, 5)
shares2 = share_shamir(t, n, 6)
shares3 = share_shamir(t, n, 7)

parties = [MultThreeParty() for _ in range(NUM_PARTIES)]

for p,s1,s2,s3 in zip(parties, shares1, shares2, shares3):
    p.round1(parties, s1, s2, s3)
for p in parties:
    p.round2()
for p in parties:
    print(p.get_view())
    assert p.output == 210

## Question 4 (10 points)

In 2-5 sentences, answer the following:

- How must we set the values of $t$ and $n$ for this protocol to work?
- Why does the protocol have this limitation?

YOUR ANSWER HERE